1. Write SQL statements to create below objects 
 - A catalog named telecom_catalog_assign </br>
 - A schema landing_zone <br>
 - A volume landing_vol <br>


In [0]:
%sql
create catalog if not exists telecom_catalog_assign;


In [0]:
%sql

create schema if not exists telecom_catalog_assign.landing_zone;


In [0]:
%sql
create volume  if not exists telecom_catalog_assign.landing_zone.landing_vol;

2. Using dbutils.fs.mkdirs, create below folders<br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/ <br>
- /Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower<br>



In [0]:
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")
dbutils.fs.mkdirs("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/")

#### Create the customer , usage and tower logs

In [0]:

customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID '''

usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

tower_logs_region = '''event_id|customer_id|tower_id|signal_strength|timestamp 
5001|101|TWR01|-80|2025-01-10 10:21:54 
5004|104|TWR05|-75|2025-01-10 11:01:12 '''



In [0]:

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", customer_csv, True)

In [0]:
# usage_tsv 
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage.tsv",usage_tsv,True)

In [0]:

dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1.csv",tower_logs_region,True)
dbutils.fs.put("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2.csv",tower_logs_region,True)


##Driver program

In [0]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("usecase3").getOrCreate();


In [0]:
dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv")

In [0]:
schema = "customer_id string,name string,age integer,city string,plan string"
df = sparkSession.read.csv(path="dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",sep=',',schema=schema)
display(df)

In [0]:
# dbutils.notebook.run('/Workspace/Users/anto2003.sfn@gmail.com/Usecase_3_solution',60)

In [0]:
usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

usage_tsv_split = usage_tsv.strip().split(" ")
# usage_tsv_split = [item.strip()

hdr = usage_tsv_split[0].split("\t")
print(type(hdr))

data = []
for item in usage_tsv_split[1:]:
    data.append(tuple(item.strip().split("\t")));
print(data);

df = spark.createDataFrame(data,hdr)
display(df)
# tower_logs_region = ''' 101




In [0]:
tower_logs_region = '''event_id|customer_id|tower_id|signal_strength|timestamp 
5001|101|TWR01|-80|2025-01-10 10:21:54 
5004|104|TWR05|-75|2025-01-10 11:01:12 '''

for item in tower_logs_region.split('\n')[1:]:
    # print(item.split(' '))
    for lst in item.strip().split(' ',2):
        print(lst.split('|'))
    # print(item.split(' ',2)[0])

In [0]:


hdr = tower_logs_region.split(" ")[0].split('|')

data = []
for item in tower_logs_region.split(' ')[1:]:
    if len(item.split('|')) >2:
        data.append(tuple(item.split('|')))
df = spark.createDataFrame(data,hdr)
display(df)




## 3. Directory Read Use Cases<br>

A. Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup<bR>

B. Demonstrate these 3 reads separately: Using pathGlobFilter Using list of paths in spark.read.csv([path1, path2]) Using .option("recursiveFileLookup","true")<br>

C.  Compare the outputs and understand when each should be used<br>

In [0]:


#Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

df = spark.read.csv(path = "dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/",sep='|',header=True,pathGlobFilter="*.csv",recursiveFileLookup=True)
display(df)


# Data cleanup needed as per Irfan's dataset

# tower_logs_region = dbutils.fs.head("dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1.csv")
# print(str)

# hdr = tower_logs_region.split(" ")[0].split('|')

# data = []
# for item in tower_logs_region.split(' ')[1:]:
#     if len(item.split('|',2)) >2:
#         data.append(tuple(item.split('|')))
# df = spark.createDataFrame(data,hdr)
# display(df)



In [0]:


# A. Read all tower logs using: Path glob filter (example: *.csv) Multiple paths input Recursive lookup

df = spark.read.csv(path = ["dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region1.csv","dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region2.csv",], sep='|', header=True, recursiveFileLookup=True)
display(df)

In [0]:
import pandas as pd
df = spark.read.csv(path = "dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/tower/region*.csv", sep='|', header=True, recursiveFileLookup=True)
display(df)


4. Schema Inference, Header, and Separator
Try the Customer, Usage files with the option and options using read.csv and format function:
header=false, inferSchema=false
or
header=true, inferSchema=true
Write a note on What changed when we use header or inferSchema with true/false?
How schema inference handled “abc” in age?

In [0]:
df =spark.read.csv(path = "dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv" , header = False, inferSchema=False
)

df.show()

# column names are defined with _c0, _c1, _c2, _c3, _c4, _c5, _c6, _c7




In [0]:
df =spark.read.csv(path = "dbfs:///Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv" , header = True,sep=','
)

display(df)

# column values are converted to column header




5. Column Renaming Usecases<br>
Apply column names using string using toDF function for customer data<br>
Apply column names and datatype using the schema function for usage data<br>
Apply column names and datatype using the StructType with IntegerType, StringType, TimestampType and other classes for towers data<br>

In [0]:
from pyspark.sql.functions import col
def getlist():
    customer_csv = ''' 101,Arun,31,Chennai,PREPAID 102,Meera,45,Bangalore,POSTPAID 103,Irfan,29,Hyderabad,PREPAID 104,Raj,52,Mumbai,POSTPAID 105,,27,Delhi,PREPAID 106,Sneha,abc,Pune,PREPAID ''';
    data = [];
    customer_csv_split = customer_csv.strip().split(" ");
    for item in customer_csv_split:
        data.append(tuple(item.split(",")));
    return data;


data = getlist();
df = sparkSession.createDataFrame(data,schema=["customer_id","name","age","city","plan"]);
display(df)


In [0]:
usage_tsv = '''customer_id\tvoice_mins\tdata_mb\tsms_count 101\t320\t1500\t20 102\t120\t4000\t5 103\t540\t600\t52 104\t45\t200\t2 105\t0\t0\t0 '''

usage_tsv_split = usage_tsv.strip().split(" ")
# usage_tsv_split = [item.strip()

hdr = usage_tsv_split[0].split("\t")
print(type(hdr))

data = []
for item in usage_tsv_split[1:]:
    data.append(tuple(item.strip().split("\t")));
print(data);

df = spark.createDataFrame(data,hdr)
display(df)
# tower_logs_region = ''' 101




In [0]:
from pyspark.sql import SparkSession
from io import StringIO
import pandas as pd # Used for an intermediate step to handle the structured text easily

# Initialize Spark Session (if not already running)
# spark = SparkSession.builder.appName("DataConversion").getOrCreate()

# # --- 1. The Raw Data ---
# data_string = """event_id|customer_id|tower_id|signal_strength|timestamp
# 5001|101|TWR01|-80|2025-01-10 10:21:54
# 5004|104|TWR05|-75|2025-01-10 11:01:12"""


data_string = '''event_id|customer_id|tower_id|signal_strength|timestamp
 5001|101|TWR01|-80|2025-01-10 10:21:54 
 5004|104|TWR05|-75|2025-01-10 11:01:12 '''

# --- 2. Convert to Spark DataFrame ---

# The most common and robust way when the data is a local string:

# a) Read the string into a Pandas DataFrame first
df_pd = pd.read_csv(StringIO(data_string), sep='|')

# b) Convert the Pandas DataFrame to a Spark DataFrame
df_spark = spark.createDataFrame(df_pd)

# --- 3. Display the Result ---
print("--- Spark DataFrame Schema ---")
df_spark.printSchema()

print("\n--- Spark DataFrame Content ---")
df_spark.show()